In [1]:
import os
from pinecone import Pinecone
from pinecone_text.sparse import BM25Encoder
from langchain_huggingface import HuggingFaceEmbeddings

def reload_rag_modules():
    import importlib
    import generator.generator as generator_module

    importlib.reload(generator_module)
    return generator_module.rag_chat, generator_module.clear_chat_history

rag_chat, clear_chat_history = reload_rag_modules()

d:\HackAWeek\Beyond-Infinity\virenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load API key from environment
from dotenv import load_dotenv
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
if not PINECONE_API_KEY:
    raise ValueError("Please set the PINECONE_API_KEY environment variable!")

INDEX_NAME = "nepali-docs-hybrid"

# Initialize Pinecone index
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)
print(f"✅ Connected to Pinecone index: {INDEX_NAME}")

# Load dense embeddings
dense_embeddings = HuggingFaceEmbeddings(model_name="universalml/Nepali_Embedding_Model")
print("✅ Dense embeddings loaded")

# Load BM25 encoder
bm25_encoder = BM25Encoder()
bm25_path = "embeddings/bm25_params.json"
if os.path.exists(bm25_path):
    bm25_encoder.load(bm25_path)
    print("✅ BM25 encoder loaded")
else:
    print(f"⚠️ BM25 parameters not found at {bm25_path}. Please generate them first!")


✅ Connected to Pinecone index: nepali-docs-hybrid
✅ Dense embeddings loaded
✅ BM25 encoder loaded


In [4]:
# Example query
# query = "मेरो नागरिकताको प्रमाणपत्र कतै हरायो, अब त्यसको अर्को कपी (प्रतिलिपि) कसरी निकाल्न सकिन्छ?"
query="अंगीकृत नागरिकता प्राप्त गर्नका लागि नेपालमा कति समयसम्म बसेको हुनुपर्छ?"
print("\n\n" + "="*80)
print("TEST 2: BALANCED HYBRID SEARCH (alpha=0.5)")
print("="*80)

result_balanced = rag_chat(
    index=index,
    query=query,
    dense_embeddings=dense_embeddings,
    bm25_encoder=bm25_encoder,
    alpha=.5,               # Balanced hybrid (50% dense, 50% sparse)
    n_retrieval=5,
    n_generation=3
)

print("\n📝 BALANCED HYBRID ANSWER (alpha=0.5):")
print(result_balanced['answer'])
print(f"\n📚 Sources found: {len(result_balanced['sources'])}")
for i, src in enumerate(result_balanced['sources'], 1):
    print(f"  {i}. {src['source_type']}: {src['source_link']}")


# Clear history before comparison test
clear_chat_history()



TEST 2: BALANCED HYBRID SEARCH (alpha=0.5)

Original Query: अंगीकृत नागरिकता प्राप्त गर्नका लागि नेपालमा कति समयसम्म बसेको हुनुपर्छ?
Language: nepali
Rewritten Query: अंगीकृत नागरिकता प्राप्त गर्नका लागि नेपालमा कति समयसम्म बसेको हुनुपर्छ?
Document Type: citizenship
Category Tag: eligibility & requirements

Generating dense vector...
Generating sparse vector (BM25)...
Performing HYBRID search (alpha=0.5) for top-5 chunks...
Retrieved 5 chunks from hybrid search

Retrieved 5 chunks for evaluation
Original retrieval order: ['47', '48', '4', '6', '15']
Loading reranker model...


d:\HackAWeek\Beyond-Infinity\virenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\adhik\.cache\huggingface\hub\models--jinaai--jina-reranker-v2-base-multilingual. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina

Reranker model loaded successfully

Reranking 5 chunks...
Original order (chunk IDs): ['47', '48', '4', '6', '15']...
Reranking complete!
Top 3 scores: ['0.0459', '-0.7852', '-1.9609']
Reranked order (chunk IDs): ['6', '4', '48', '15', '47']...
 Reranking applied!
Reranked order: ['6', '4', '48', '15', '47']

Using top 3 chunks for answer generation
Generation chunk IDs: ['6', '4', '48']

Final context chunks prepared for RAG prompt:
--- Chunk 1 ---
Chunk ID: 6...
Score: 0.0459
Source: Citizenship_Act
Preview: Base Chunk:
(४) देहायका शर्त र अवस्था पूरा गरिसकेका विदेशी नागरिकमध्ये विज्ञान, दर्शनशास्त्र, कला, साहित्य, विश्वशान्ति, मानव कल्याण वा नेपालको औद्योग...

--- Chunk 2 ---
Chunk ID: 4...
Score: -0.7852
Source: Citizenship_Act
Preview: Base Chunk:
४. जन्मको आधारमा नेपाली नागरिकता प्राप्ति : (१) सम्वत् २०४६ साल चैत्र मसान्तसम्म नेपाल सरहदभित्र जन्म भई नेपालमा स्थायी रुपले बसोबास गर्दै...

--- Chunk 3 ---
Chunk ID: 48...
Score: -1.9609
Source: Citizenship_Faqs
Preview: Base Chunk:
प्